# AWS Cognito Client Example
In this notebook, we'll demonstrate how you, as a programmatic client, can call AWS Cognito to receive an appropriate JSON web token (JWT) to represent that you have the appropriate security and authorization. We'll stop short of demonstrating how we would pass this JWT to the secured application we're looking to invoke.

## Notebook Setup
In this brief section, we'll set ourselves up for success by importing the required Python packages and load up sensitive credentials from a local `.env` file. Using a `.gitignore` file, you will not see this file in GitHub. In terms of this notebook, the only sensitive credential we will be using is the Cognito client secret.

In [1]:
# Importing the necessary Python libraries
import os
import jwt
import json
import base64
import requests
from dotenv import load_dotenv

# Loading sensitive credentials from a local .env file (Not uploaded to GitHub!)
load_dotenv("../.env")

True

## Cognito Setup
We'll now establish all our information we'll need to appropriately invoke Cognito. Please note: this information will be proprietary to YOU! The code I am testing with will no longer work even for myself as I am continuously rebuilding / redestroying the AWS Cognito infrastructure. If you would like to see Terraform to build the Cognito resources, including the Cognito client we are using here, please see the Terraform code under `/terraform/aws` directory at the root of this repo.

In [2]:
# Setting all the Cognito information
COGNITO_DOMAIN = "https://my-example-286574326306-domain.auth.us-east-1.amazoncognito.com/oauth2/token"
COGNITO_CLIENT_ID = '2mmjvk1nrdss8264qn1igthm4k'
COGNITO_CLIENT_SECRET = os.environ['COGNITO_CLIENT_SECRET']
COGNITO_RESOURCE_SERVER_ID = "my-api"

# Setting the scopes we want to apply
SCOPES = [
    f"{COGNITO_RESOURCE_SERVER_ID}/read",
    f"{COGNITO_RESOURCE_SERVER_ID}/write",
]
SCOPE = ' '.join(SCOPES)

In [3]:
# Encoding the Cognito client ID / secret
encoded_secret = base64.b64encode(
    f"{COGNITO_CLIENT_ID}:{COGNITO_CLIENT_SECRET}".encode("utf-8")
).decode("utf-8")

# Setting the headers we will pass to Cognito
headers = {
    "Authorization": f"Basic {encoded_secret}",
    "Content-Type": "application/x-www-form-urlencoded",
}

# Setting the scopes data we will pass to Cognito
scopes_data = {
    "grant_type": "client_credentials",
    "scope": SCOPE,
}

## Getting Credentials from Cognito
Now that we have set up what we need to get our credentials in AWS Cognito, we are ready to retrieve those credentials from Cognito!

In [4]:
# Getting the appropriate authroization credentials from Cognito
response = requests.post(
    COGNITO_DOMAIN,
    headers = headers,
    data = scopes_data,
    timeout = 30
)

# Transforming the response into a JSON format
response_json = response.json()

In [5]:
# Getting the JWT token from the response
token = response_json['access_token']

# Decoding the JWT token
decoded_token = jwt.decode(token, options = {"verify_signature": False})

# Printing the decoded JWT token in a readable format
print(json.dumps(decoded_token, indent = 4))

{
    "sub": "2mmjvk1nrdss8264qn1igthm4k",
    "token_use": "access",
    "scope": "my-api/write my-api/read",
    "auth_time": 1771550873,
    "iss": "https://cognito-idp.us-east-1.amazonaws.com/us-east-1_S0K7gm11O",
    "exp": 1771554473,
    "iat": 1771550873,
    "version": 2,
    "jti": "bebefb93-167f-477b-8f78-3e2d8dd1e2fe",
    "client_id": "2mmjvk1nrdss8264qn1igthm4k"
}
